# WSI 2022L
# Laboratorium 4 (ID3)
# Michał Brus, 299106

## Importy

In [404]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Wczytanie danych

In [405]:
train_data = pd.read_csv('breast-cancer.data')
train_data.head(3)

,event,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no


## Funkcja licząca entropię

In [406]:
def calc_entropy(data, classificator_name, classificator_values):
    rows_count = data.shape[0]
    entropy = 0

    for c_value in classificator_values:
        c_count = data[data[classificator_name] == c_value].shape[0]
        c_entropy = 0
        if c_count != 0:
            c_probability = c_count / rows_count
            c_entropy = -c_probability * np.log2(c_probability)
        entropy += c_entropy

    return entropy

### Całkowita entropia danych

In [407]:
total_entropy = calc_entropy(train_data, 'irradiat', train_data['irradiat'].unique())
total_entropy

0.7912991129798124

## Funcja licząca Information Gain

In [408]:
def calc_inf_gain(feature_name, data, classificator_name, classificator_values):
    feature_values = data[feature_name].unique()
    rows_count = data.shape[0]
    feature_inf_gain = 0

    for f_value in feature_values:
        f_value_data = data[data[feature_name] == f_value]
        f_value_count = f_value_data.shape[0]
        f_value_entropy = calc_entropy(f_value_data, classificator_name, classificator_values)
        f_value_probability = f_value_count / rows_count
        feature_inf_gain += f_value_probability * f_value_entropy

    return calc_entropy(data, classificator_name, classificator_values) - feature_inf_gain


### Przykładowe IG dla 'tumor-size'

In [409]:
calc_inf_gain('tumor-size', train_data, 'irradiat', train_data['irradiat'].unique())

0.03697234523828685

## Funkcja znajdująca zmienną, która ma największy IG

In [410]:
def find_max_inf_feature(data, classificator_name, classificator_values):
    features = data.columns.drop(classificator_name)
    max_inf_gain = -1
    max_inf_feature = None

    for feature in features:
        feature_inf_gain = calc_inf_gain(feature, data, classificator_name, classificator_values)
        if max_inf_gain < feature_inf_gain:
            max_inf_gain = feature_inf_gain
            max_inf_feature = feature

    return max_inf_feature

In [411]:
find_max_inf_feature(train_data, 'irradiat', train_data['irradiat'].unique())

'inv-nodes'

## Funkcja pomocnicza do generacji drzewa

In [412]:
def generate_subtree(feature_name, data, classififcator_name, classificator_values):
    feature_dict = data[feature_name].value_counts(sort=False)
    tree = {}

    for f_value, f_value_count in feature_dict.iteritems():
        f_value_data = data[data[feature_name] == f_value]
        is_one_value_node = False

        for c_value in classificator_values:
            c_value_count = f_value_data[f_value_data[classififcator_name] == c_value].shape[0]
            if c_value_count == f_value_count:
                tree[f_value] = c_value
                data = data[data[feature_name] != f_value]
                is_one_value_node = True

        if not is_one_value_node:
            tree[f_value] = '?'

    return tree, data

## Procedura generująca drzewo

In [413]:
def generate_tree(root, previous_feature_value, data, classificator_name, classificator_values, max_depth, current_depth=0):
    if data.shape[0] != 0 and current_depth < max_depth:
        current_depth += 1
        max_inf_feature = find_max_inf_feature(data, classificator_name, classificator_values)
        tree, data = generate_subtree(max_inf_feature, data, classificator_name, classificator_values)
        next_root = None

        if previous_feature_value is not None:
            root[previous_feature_value] = {}
            root[previous_feature_value][max_inf_feature] = tree
            next_root = root[previous_feature_value][max_inf_feature]
        else:
            root[max_inf_feature] = tree
            next_root = root[max_inf_feature]

        for feature_value, branch in list(next_root.items()):
            if branch == '?':
                f_value_data = data[data[max_inf_feature] == feature_value]
                generate_tree(next_root, feature_value, f_value_data, classificator_name, classificator_values, max_depth, current_depth)


## Główny algorytm (ID3) ze zmienną głębokością

In [414]:
def id3(train_data, classificator_name, max_depth):
    data = train_data.copy()
    tree = {}
    classificator_values = data[classificator_name].unique()
    generate_tree(tree, None, data, classificator_name, classificator_values, max_depth)
    return tree

## Funkcja predykcji

In [415]:
def find_prediction(tree, row):
    if not isinstance(tree, dict):
        return tree
    else:
        current_feature = next(iter(tree))
        feature_value = row[current_feature]
        if feature_value in tree[current_feature]:
            return find_prediction(tree[current_feature][feature_value], row)
        else:
            return '?'

def predict(tree, data):
    predictions = []
    for i, row in data.iterrows():
        predictions.append(find_prediction(tree, row))
    return predictions

## Podział danych na zbiór testowy i trenujący. Generacja drzewa i predykcja rezultatów. Sprawdzenie dokładności działania.

In [416]:
def test():
    train_data = pd.read_csv('breast-cancer.data')
    X = train_data.iloc[:, :-1]
    y = train_data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    xy_train = X_train.assign(irradiat=y_train)
    xy_test = X_test.assign(irradiat=y_test)

    tree = id3(xy_train, 'irradiat', 5)
    predictions = predict(tree, xy_test)
    accuracy = metrics.accuracy_score(y_test, predictions)
    print(f"Actual values: {list(y_test)}")
    print(f"Predictions: {predictions}")
    print(f"Accuracy: {accuracy}")

In [417]:
test()

Actual values: ['no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no']
Predictions: ['no', 'no', '?', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', '?', 'no', 'no', 'yes', 'no', 'yes', 'no']
Accuracy: 0.6724137931034483
